In [1]:
!pip install FPDF

  Preparing metadata (setup.py) ... done
  Created wheel for FPDF: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=d62668e27252abab78906dcbba12727aa3827ea69450524db78bb7178fa23197
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built FPDF


In [ ]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from itertools import product
import requests
from fpdf import FPDF

# Load the clinical dataset
clinical_data = pd.read_excel('clinical.xlsx')

# Map categorical GI values to numerical values in clinical dataset
gi_mapping = {'Low': 1, 'Moderate': 2, 'High': 3}
clinical_data['GI'] = clinical_data['GI'].map(gi_mapping)

# Fill missing values with the mean
clinical_data['GI'] = clinical_data['GI'].fillna(clinical_data['GI'].mean())

# Concatenate relevant columns into a single feature
clinical_data['features'] = clinical_data.astype(str).agg(' '.join, axis=1)

# Initialize the TF-IDF Vectorizer for clinical data
tfidf_vectorizer_clinical = TfidfVectorizer()

# Fit and transform the feature for clinical data
tfidf_matrix_clinical = tfidf_vectorizer_clinical.fit_transform(clinical_data['features'])

# Load the food dataset
food_data = pd.read_excel('Food Dataset.xlsx')

# Map categorical GI values to numerical values in food dataset
food_data['GI'] = food_data['GI'].map(gi_mapping)

# Fill missing values with an empty string
food_data['GI'] = food_data['GI'].fillna('')

# Convert 'GI' column to string type
food_data['GI'] = food_data['GI'].astype(str)

# Concatenate relevant columns into a single feature
food_data['features'] = food_data['Meal Type'] + ' ' + food_data['Preference'] + ' ' + food_data['Allergy'] + ' ' + food_data['GI']

# Initialize the TF-IDF Vectorizer for food data
tfidf_vectorizer_food = TfidfVectorizer(vocabulary=tfidf_vectorizer_clinical.vocabulary_)

# Fit and transform the feature for food data
tfidf_matrix_food = tfidf_vectorizer_food.fit_transform(food_data['features'])

# Calculate cosine similarity between clinical data and food data
cosine_similarities = cosine_similarity(tfidf_matrix_clinical, tfidf_matrix_food)

# Get indices of similar dishes for each clinical data point
similar_indices_clinical = [list(similarities.argsort()[-20:][::-1]) for similarities in cosine_similarities]

def validate_input(prompt, options):
    while True:
        user_input = input(prompt).strip()
        if user_input in options:
            return user_input
        else:
            print("Invalid input. Please try again.")

def recommend_dishes(age, gender, hba1c_levels, exercise, glucose_level, allergy, preference, meal_type):
    # Construct user input
    user_input = f"{age} {gender} {hba1c_levels} {exercise} {glucose_level} {allergy} {preference} {meal_type}"

    # Transform user input using TF-IDF Vectorizer
    user_tfidf = tfidf_vectorizer_clinical.transform([user_input])

    # Calculate cosine similarity between user input and all dishes in food dataset
    similarity_scores_food = cosine_similarity(user_tfidf, tfidf_matrix_food)

    # Get indices of similar dishes
    similar_indices_food = similarity_scores_food.argsort()[0][-20:][::-1]  # Top 20 similar dishes from food dataset

    # Group dishes by meal type and preference
    grouped_dishes = defaultdict(list)
    for idx in similar_indices_food:
        dish = food_data.iloc[idx]
        grouped_dishes[(dish['Preference'], dish['Meal Type'])].append(idx)

    # Filter dishes based on user preferences, meal type, and allergy
    filtered_dishes = []
    for pref, m_type, idx in product([preference, 'No Preference'], [meal_type, 'No Meal Type'], similar_indices_food):
        if (pref, m_type) not in grouped_dishes:
            continue
        dish = food_data.iloc[idx]
        if allergy != 'No Allergy' and allergy in dish['Allergy']:
            continue
        if pref == 'No Preference' or m_type == 'No Meal Type':
            filtered_dishes.extend(grouped_dishes[(pref, m_type)])
        else:
            if pref == 'Vegan' and ('NonVeg' in dish['Preference'] or 'Egg' in dish['Allergy']):
                continue  # Skip non-veg and egg dishes for vegan preference
            elif pref == 'Jain' and ('NonVeg' in dish['Preference'] or 'Egg' in dish['Allergy']):
                continue  # Skip non-veg and egg dishes for Jain preference
            elif pref == 'Veg' and ('NonVeg' in dish['Preference'] or 'Egg' in dish['Allergy']):
                continue  # Skip non-veg and egg dishes for vegetarian preference
            filtered_dishes.append(idx)

    # Select diverse set of dishes from filtered options
    recommended_indices = select_diverse_items(filtered_dishes, num_items=3)

    # If no matching dishes found, select random dishes
    if not recommended_indices:
        all_indices = [idx for sublist in grouped_dishes.values() for idx in sublist]
        recommended_indices = random.sample(all_indices, min(3, len(all_indices)))

    # Get recommendations from selected dishes
    recommended_dishes_info = food_data.iloc[recommended_indices][['Dish Name', 'Carbs', 'Protein', 'Fats', 'Fiber', 'Calories', 'Image']].values.tolist()

    return recommended_dishes_info

def select_diverse_items(indices, num_items):
    # Select diverse set of items using pairwise cosine similarity
    selected_indices = []
    if len(indices) <= num_items:
        return indices
    pairwise_similarity = cosine_similarity(tfidf_matrix_food[indices])
    selected_indices.append(indices.pop(0))
    while len(selected_indices) < num_items:
        max_sim = 0
        max_idx = None
        for i, idx in enumerate(indices):
            sim = min(pairwise_similarity[i, selected_indices])
            if sim > max_sim:
                max_sim = sim
                max_idx = i
        selected_indices.append(indices.pop(max_idx))
    return selected_indices

# Function to download image from URL and save it locally
def download_image(image_url, filename):
    response = requests.get(image_url)
    with open(filename, 'wb') as f:
        f.write(response.content)

# Function to generate PDF with recommended dishes and images
def generate_pdf(recommended_dishes):
    pdf = FPDF()
    pdf.add_page()

    # Define image width and height
    image_width = 50
    image_height = 30

    # Loop through recommended dishes and add image and text to the PDF
    for dish in recommended_dishes:
        dish_name = dish[0]
        image_url = dish[6]  # Assuming the image URL is stored in the 7th index
        image_filename = dish_name.replace(' ', '_') + '.jpg'  # Create a unique filename for each image
        download_image(image_url, image_filename)

        # Add text on the left side
        pdf.set_font("Arial", size=12)
        pdf.cell(100, 8, txt=f"Dish Name: {dish_name}", ln=True)
        pdf.cell(100, 8, txt=f"Carbs: {dish[1]}", ln=True)
        pdf.cell(100, 8, txt=f"Protein: {dish[2]}", ln=True)
        pdf.cell(100, 8, txt=f"Fats: {dish[3]}", ln=True)
        pdf.cell(100, 8, txt=f"Fiber: {dish[4]}", ln=True)
        pdf.cell(100, 8, txt=f"Calories: {dish[5]}", ln=True)

        # Add image on the right side aligned with the text
        pdf.image(image_filename, x=pdf.get_x() + 110, y=pdf.get_y() - 50, w=image_width, h=image_height)

        pdf.ln(max(image_height, 50) + 5)  # Add space after the image or text

    # Save the PDF
    pdf.output("recommended_dishes.pdf")

# Example usage with input validation
age_options = ['1', '2', '3', '4']
gender_options = ['1', '2']
hba1c_levels_options = ['1', '2', '3']
exercise_options = ['1', '2', '3']
glucose_level_options = ['1', '2', '3']
allergy_options = ['1', '2', '3', '4', '5', '6']
preference_options = ['1', '2', '3', '4']
meal_type_options = ['1', '2', '3']

age = validate_input("Enter your age(1 for 15-30,2 for 31-45,3 for 46-60,4 for 60 above): ", age_options)
gender = validate_input("Enter your gender (1 for Male, 2 for Female): ", gender_options)
hba1c_levels = validate_input("Enter your HbA1c level (1 for Below 5.7%, 2 for 5.7% to 6.4%, 3 for 6.5% or higher): ", hba1c_levels_options)
exercise = validate_input("Enter your exercise frequency (1 for Daily, 2 for Moderate (3-4 days/week), 3 for No Exercise): ", exercise_options)
glucose_level = validate_input("How often does your glucose level go high? (1 for Frequently, 2 for Often, 3 for Rarely): ", glucose_level_options)
allergy = validate_input("Enter your allergy (1 for Gluten, 2 for Dairy, 3 for Nuts, 4 for Seafood, 5 for Egg, 6 for No Allergy): ", allergy_options)
preference = validate_input("Enter your preference (1 for Veg, 2 for NonVeg, 3 for Vegan, 4 for Jain): ", preference_options)
meal_type = validate_input("Enter meal type (1 for Breakfast, 2 for Lunch, 3 for Dinner): ", meal_type_options)

recommended_dishes = recommend_dishes(age, gender, hba1c_levels, exercise, glucose_level, allergy, preference, meal_type)
print("\nRecommended Dishes:")
for dish in recommended_dishes:
    print("Dish Name:", dish[0])
    print("Carbs:", dish[1])
    print("Protein:", dish[2])
    print("Fats:", dish[3])
    print("Fiber:", dish[4])
    print("Calories:", dish[5])
    print("Image:", dish[6])
    print()

generate_pdf(recommended_dishes)

